In [1]:
!pip install -r requirements.txt
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install flash-attn==2.5.8 --no-build-isolation 
!pip install pip install accelerate

Looking in indexes: https://pypi.org/simple/
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "THUDM/glm-4v-9b"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
).eval()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

## single pic

In [9]:
%%time

import time

from PIL import Image
from io import BytesIO
import base64
import json

start_time = time.time() 

query = '识别图中文本'

device = "cuda:0"
image = Image.open("number_pics/merged_爆炸图_双桶_60E02系列_下排水_双进水_桶壁一体-解锁新/1.jpg").convert('RGB')

buffer = BytesIO()
image.save(buffer, format="JPEG")
im_bytes = buffer.getvalue()
im_b64 = base64.b64encode(im_bytes).decode('utf-8')

tmp = json.dumps({"image": im_b64})
_tmp = json.loads(tmp)
image = _tmp["image"]
image = Image.open(BytesIO(base64.b64decode(image)))

inputs = tokenizer.apply_chat_template([{"role": "user", "image": image, "content": query}],
                                       add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                       return_dict=True)  # chat mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    end_time0 = time.time()
    # 计算程序运行时间
    execution_time = end_time0 - start_time
    print(f"glm程序运行时间 到encoder: {execution_time:.6f} 秒")

    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    
    end_time1 = time.time()
    # 计算程序运行时间
    execution_time = end_time1 - start_time
    print(f"glm程序运行时间 到decoder: {execution_time:.6f} 秒")

    print(tokenizer.decode(outputs[0]))

glm程序运行时间 到encoder: 8.656724 秒
glm程序运行时间 到decoder: 8.656841 秒
图中文本为：

0 1. 2

1 4. 1

2 1. 16

3 2. 3

4 1. 5

5 2. 2

6 1. 9

7 4. 6

8 1. 4

9 2. 1

10 4. 2

11 2. 10

12 4. 3

13 2. 4

14 4. 8

15 1. 14

16 4. 10

17 2. 8

18 1. 3

19 3. 1 <|endoftext|>
CPU times: user 9.86 s, sys: 269 ms, total: 10.1 s
Wall time: 8.66 s


## multi 

In [12]:
%%time

from PIL import Image
from io import BytesIO
import base64
import json

def process(path):
    image = Image.open(path).convert('RGB')

    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    im_bytes = buffer.getvalue()
    im_b64 = base64.b64encode(im_bytes).decode('utf-8')

    tmp = json.dumps({"image": im_b64})
    _tmp = json.loads(tmp)
    image = _tmp["image"]
    image = Image.open(BytesIO(base64.b64decode(image)))

    return image

start_time = time.time() 

query='''图中是一个表格，第一列是序号，第二列是图片，按照从上往下的顺序输出以下内容：[[序号,图片中的数字],...],如果图片中没有数字则写为[[序号,"None"]...]'''


#prompt='''图中是一个3列的表格，第一列是序号，第二列是图片，第三列是序号，第四列是图片，第五列是序号，第六列是图片，按照先从上往下再从左往右的顺序输出以下内容：[[序号,图片中的数字],...],如果图片中没有数字则写为[[序号,"None"]...]'''
#prompt = '''自上而下从左到右识别图中圆圈中数字'''

device = "cuda:0"
image0 = process('number_pics/merged_爆炸图_双桶_60E02系列_下排水_双进水_桶壁一体-解锁新/1.jpg')
image1 = process('number_pics/merged_爆炸图_双桶_60E02系列_下排水_双进水_桶壁一体-解锁新/0.jpg')

batch_message = [
        [
            {"role": "user", "image": image0, "content": query}
        ],
        [
            {"role": "user", "image": image1, "content": query}
        ],[
            {"role": "user", "image": image0, "content": query}
        ],
    [
            {"role": "user", "image": image0, "content": query}
        ]
    ]
inputs = tokenizer.apply_chat_template(batch_message,
                                       add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                       return_dict=True)  # chat mode
inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    end_time0 = time.time()
    # 计算程序运行时间
    execution_time = end_time0 - start_time
    print(f"glm程序运行时间 到encoder: {execution_time:.6f} 秒")
    
    responses = []
    for input_ids, output_ids in zip(inputs.input_ids, outputs):
        response = tokenizer.decode(output_ids[len(input_ids):], skip_special_tokens=True).strip()
        
        responses.append(response)
        
    end_time1 = time.time()
    # 计算程序运行时间
    execution_time = end_time1 - start_time
    print(f"glm程序运行时间 到decoder: {execution_time:.6f} 秒")


glm程序运行时间 到encoder: 21.654087 秒
glm程序运行时间 到decoder: 21.664298 秒
CPU times: user 23.3 s, sys: 807 ms, total: 24.1 s
Wall time: 21.7 s


In [13]:
for response in responses:
    print("=" * 10)
    print(response)

[[0,1.2],[1,4.1],[2,1.16],[3,2.3],[4,1.5],[5,2.2],[6,1.9],[7,4.6],[8,1.4],[9,2.1],[10,4.2],[11,2.10],[12,4.3],[13,2.4],[14,4.8],[15,1.14],[16,4.10],[17,2.8],[18,1.3],[19,3.1]]
[[0,2.11],[1,2.5],[2,4.4],[3,1.10],[4,1.18],[5,1.12],[6,1.1],[7,1.15],[8,2],[9,1.11],[10,4.13],[11,4.11],[12,1.8],[13,4.12],[14,4.9],[15,1.13],[16,1.7],[17,3.2],[18,1.17],[19,1.6]]
[[0,1.2],[1,4.1],[2,1.16],[3,2.3],[4,1.5],[5,2.2],[6,1.9],[7,4.6],[8,1.4],[9,2.1],[10,4.2],[11,2.10],[12,4.3],[13,2.4],[14,4.8],[15,1.14],[16,4.10],[17,2.8],[18,1.3],[19,3.1]]
[[0,1.2],[1,4.1],[2,1.16],[3,2.3],[4,1.5],[5,2.2],[6,1.9],[7,4.6],[8,1.4],[9,2.1],[10,4.2],[11,2.10],[12,4.3],[13,2.4],[14,4.8],[15,1.14],[16,4.10],[17,2.8],[18,1.3],[19,3.1]]
